In [1]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
# graficos incrustados
%matplotlib inline

#Carga de los CSV
BBVA_data = pd.read_csv('labelencoder_train_clientes.csv', header=0)
print("Data Train: ", BBVA_data.shape)
BBVA_test = pd.read_csv('labelencoder_test_clientes.csv', header=0)
print("Data Test:  ", BBVA_test.shape)

Data Train:  (70000, 54)
Data Test:   (30000, 53)


In [2]:
primer = list(BBVA_data.columns.values)[0]
primer

'Unnamed: 0'

In [3]:
X_train = BBVA_data.drop([primer, 'ID_CORRELATIVO', 'ATTRITION'], axis=1)
print(type(X_train), X_train.shape)
X_predic = BBVA_test.drop([primer, 'ID_CORRELATIVO'], axis=1)
print(type(X_predic), X_predic.shape)
y_train = BBVA_data['ATTRITION'].values
print(type(y_train), y_train.shape)

<class 'pandas.core.frame.DataFrame'> (70000, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)
<class 'numpy.ndarray'> (70000,)


In [4]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
extra = ExtraTreesClassifier(n_estimators=250, n_jobs=2, min_samples_split=25, 
                             max_features=0.99, random_state=2)
columnas = list(X_train.columns.values)
era = RFE(extra, 15)  # número de atributos a seleccionar
era = era.fit(X_train.values, y_train)
# imprimir resultados
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
print(atributos)

['RANG_INGRESO', 'EDAD', 'ANTIGUEDAD', 'RANG_SDO_PASIVO_MENOS0', 'FLG_SEGURO_MENOS2', 'RANG_NRO_PRODUCTOS_MENOS0', 'NRO_ACCES_CANAL3_MENOS0', 'NRO_ACCES_CANAL3_MENOS1', 'NRO_ACCES_CANAL3_MENOS2', 'NRO_ACCES_CANAL3_MENOS3', 'NRO_ENTID_SSFF_MENOS0', 'NRO_ENTID_SSFF_MENOS1', 'NRO_ENTID_SSFF_MENOS2', 'NRO_ENTID_SSFF_MENOS5', 'FLG_SDO_OTSSFF_MENOS0']


In [5]:
X_train = X_train.loc[:,atributos].values
print(type(X_train), X_train.shape)
X_predic = X_predic.loc[:,atributos].values
print(type(X_predic), X_predic.shape)

<class 'numpy.ndarray'> (70000, 15)
<class 'numpy.ndarray'> (30000, 15)


In [6]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler(copy=True, with_mean=True, with_std=True)
xx_train_std = sc_X.fit_transform(X_train.astype(float))
print(type(xx_train_std), xx_train_std.shape)
xx_predic_std = sc_X.transform(X_predic.astype(float))
print(type(xx_predic_std), xx_predic_std.shape)

from sklearn.preprocessing import MinMaxScaler
sc_MM = MinMaxScaler()
xx_train_mm = sc_MM.fit_transform(X_train.astype(float))
print(type(xx_train_mm), xx_train_mm.shape)
xx_predic_mm = sc_MM.transform(X_predic.astype(float))
print(type(xx_predic_mm), xx_predic_mm.shape)
idd = BBVA_test['ID_CORRELATIVO']
idd.shape

<class 'numpy.ndarray'> (70000, 15)
<class 'numpy.ndarray'> (30000, 15)
<class 'numpy.ndarray'> (70000, 15)
<class 'numpy.ndarray'> (30000, 15)


(30000,)

In [7]:
initial = '33_15_label_imputacion_factorizacion_15var_'
import csv
def generate_csv(clf, predic, name):
    name = initial + name
    result = clf.predict_proba(predic)
    result = [float("{0:.5f}".format(_)) for _ in list(zip(*result))[1]]
    result = list(zip(list(idd), result))
    result.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
    with open("{}.csv".format(name), 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in result:
           wr.writerow(row)

In [8]:
list_train = [(xx_train_std, xx_predic_std), (xx_train_mm, xx_predic_mm)]

for ixx, train in enumerate(list_train):
    red = MLPClassifier(max_iter=1000000000, hidden_layer_sizes=(100,100), alpha=0.0000000001, solver='sgd', random_state=2)
    red.fit(train[0], y_train)
    name ='{0}_RedNeuronal_{1}'.format(
        'STD2' if ixx == 0 else 'MINMAX2', 'sgd')
    generate_csv(red, train[1], name)
    print("10varLABELENCODER///////////////////////////////////////////////{0}".format(name))

15varLABELENCODER///////////////////////////////////////////////STD2_RedNeuronal_sgd
15varLABELENCODER///////////////////////////////////////////////MINMAX2_RedNeuronal_sgd


In [10]:
sum(y_train)


10822

In [8]:
list_train = [(xx_train_std, xx_predic_std), (xx_train_mm, xx_predic_mm)]
from sklearn.ensemble import RandomForestClassifier
for ixx, train in enumerate(list_train):
    red =  RandomForestClassifier(n_estimators=250, n_jobs=2, min_samples_split=25,
                                      max_features=0.99, random_state=2)
    red.fit(train[0], y_train)
    name ='{0}_RandomForest_{1}'.format(
        'STD2' if ixx == 0 else 'MINMAX2', 'arboles250_sp25')
    generate_csv(red, train[1], name)
    print("15varLABELENCODER///////////////////////////////////////////////{0}".format(name))

15varLABELENCODER///////////////////////////////////////////////STD2_RandomForest_arboles250_sp25
15varLABELENCODER///////////////////////////////////////////////MINMAX2_RandomForest_arboles250_sp25
